In [17]:
import pkg_resources
from packaging import version
import re
from typing import Dict, Tuple, List
import subprocess
import sys

def parse_requirement_line(line: str) -> Tuple[str, str, str]:
    """Parse a single requirement line into package name, operator, and version."""
    line = line.strip()
    if not line or line.startswith('#'):
        return None
    
    # Handle the different requirement formats
    pattern = r'^([^<>=!~]+)(>=|<=|!=|==|~=|>|<)(.+)$'
    match = re.match(pattern, line)
    
    if not match:
        # Handle case with no operator (implies latest version)
        package = line
        return (package.strip(), '==', 'latest')
        
    package, operator, version_str = match.groups()
    return (package.strip(), operator.strip(), version_str.strip())

def get_installed_version(package: str) -> str:
    """Get the installed version of a package."""
    try:
        return pkg_resources.get_distribution(package).version
    except pkg_resources.DistributionNotFound:
        return None

def compare_versions(installed: str, required: str, operator: str) -> bool:
    """Compare installed version against requirement."""
    if required == 'latest':
        return True
        
    try:
        installed_ver = version.parse(installed)
        required_ver = version.parse(required)
        
        ops = {
            '==': lambda i, r: i == r,
            '>=': lambda i, r: i >= r,
            '<=': lambda i, r: i <= r,
            '>': lambda i, r: i > r,
            '<': lambda i, r: i < r,
            '!=': lambda i, r: i != r,
            '~=': lambda i, r: i >= r and i.release[0] == r.release[0]
        }
        
        return ops[operator](installed_ver, required_ver)
    except version.InvalidVersion:
        return False

def check_requirements(requirements_file: str) -> Dict[str, dict]:
    """
    Check installed packages against requirements file.
    
    Args:
        requirements_file: Path to requirements.txt file
        
    Returns:
        Dictionary with results for each package:
        {
            'package_name': {
                'installed_version': str,
                'required_version': str,
                'operator': str,
                'matches': bool
            }
        }
    """
    results = {}
    
    try:
        with open(requirements_file, 'r') as f:
            for line in f:
                parsed = parse_requirement_line(line)
                if not parsed:
                    continue
                    
                package, operator, required_version = parsed
                installed_version = get_installed_version(package)
                
                results[package] = {
                    'installed_version': installed_version,
                    'required_version': required_version,
                    'operator': operator,
                    'matches': (
                        installed_version is not None and
                        compare_versions(installed_version, required_version, operator)
                    )
                }
                
    except FileNotFoundError:
        raise FileNotFoundError(f"Requirements file not found: {requirements_file}")
    
    return results


In [18]:
requirements_file_path = '../requirements_old.txt'

results = check_requirements(requirements_file=requirements_file_path)

# Print python version
print(f"Python version: {sys.version.split()[0]}") # 3.10.12

for package, info in results.items():
    status = "✓" if info['matches'] else "✗"
    print(f"{status} {package}:")
    print(f"    Installed: {info['installed_version']}")
    print(f"    Required: {info['operator']} {info['required_version']}")
    print()
    

Python version: 3.10.12
✓ nibabel:
    Installed: 5.3.2
    Required: >= 4.0.0

✓ numpy:
    Installed: 1.26.4
    Required: >= 1.21.0

✓ tqdm:
    Installed: 4.67.1
    Required: >= 4.65.0

✓ dvc:
    Installed: 3.59.0
    Required: >= 3.0.0

✓ dvc[gdrive] # for Google Drive support:
    Installed: 3.59.0
    Required: == latest

✓ PyYAML:
    Installed: 6.0.2
    Required: >= 6.0.0

✓ scikit-image:
    Installed: 0.25.0
    Required: >= 0.19.0

✓ pandas:
    Installed: 2.2.3
    Required: >= 1.5.0

✓ boto3:
    Installed: 1.36.1
    Required: >= 1.26.0

✓ SimpleITK:
    Installed: 2.4.1
    Required: >= 2.2.0



In [19]:
from datetime import datetime


def create_version_locked_requirements(requirements_file: str, output_file: str) -> None:
    """
    Create a new requirements file with exact versions of currently installed packages.
    
    Args:
        requirements_file: Path to input requirements.txt
        output_file: Path where to save the new requirements file
    """
    results = check_requirements(requirements_file)
    
    with open(output_file, 'w') as f:
        # Add a comment with the current date and python version
        f.write(f"# Generated on {datetime.now().isoformat()}\n")
        f.write(f"# Python version: {sys.version.split()[0]}\n\n")
        for package, info in results.items():
            if info['installed_version']:
                f.write(f"{package}=={info['installed_version']}\n")
            else:
                f.write(f"# {package}: not installed\n")

In [20]:
create_version_locked_requirements(
    requirements_file=requirements_file_path,
    output_file=requirements_file_path.replace('_old.txt', '.txt')
)